### Imports

In [2]:
# processing
from operator import methodcaller
import csv
import re
import numpy as np
import pandas as pd
from pprint import pprint
import string


# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt


#sci-kit
from sklearn import feature_extraction

C:\Users\pjz1\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Processing

In [5]:
blacklist = [t.strip() for t in next(csv.reader(open("blacklist.csv", 'r')))]
levels = [1, 2, 3]

inPath = "raw.csv"

inFile = open(inPath, 'r')
inReader = csv.reader(inFile)

docTokens = dict()


next(inReader)
for inRow in inReader:
    term = inRow[0]
    sentence = inRow[2]
    docID = inRow[3]
    
    token = "_".join([t for t in term.split(":") if re.match(r'[^\W\d]*$', t) and not t in blacklist])
    
    level = token.count("_")
    
    if level in levels and not token in blacklist and len(token) > 0:
        if docID in docTokens:
            docTokens[docID] += [token]
        else:
            docTokens[docID] = [token]

docIDs = list(docTokens.keys())
data = list(docTokens.values())

### LDA Modeling

In [6]:
dictionary = corpora.Dictionary(data)
texts = data

corpus = [dictionary.doc2bow(text) for text in texts]

In [7]:
numTops = 20

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                       id2word=dictionary,
                                       num_topics=numTops, 
                                       random_state=100,
                                       update_every=1,
                                       chunksize=8,
                                       passes=1,
                                       alpha='auto',
                                       per_word_topics=True)


print(lda_model.print_topics())

#select dist topic
#for each dist id
#where p > bound
#for each term
#check if it's in previous list

doc_lda = lda_model[corpus]

[(0, '0.055*"crystal_structure" + 0.032*"water_molecule" + 0.029*"middot_middot" + 0.015*"hydrogen_bond" + 0.010*"stack_interaction" + 0.009*"rotation_axis" + 0.009*"uncoordinated_water_molecule" + 0.009*"perchlorate_anion" + 0.008*"c_n_bond" + 0.007*"half_occupancy"'), (1, '0.029*"unit_cell" + 0.023*"x_ray_diffraction" + 0.023*"b_axis" + 0.022*"small_angle" + 0.021*"inversion_centre" + 0.018*"c_bond" + 0.013*"o_interaction" + 0.008*"symmetry_centre" + 0.008*"dimensional_structure" + 0.007*"basal_plane"'), (2, '0.009*"ring_system" + 0.009*"special_position" + 0.007*"saxs_intensity" + 0.007*"thermal_expansion" + 0.007*"membrane_protein" + 0.005*"transmembrane_protein" + 0.005*"respectively_deg_dihedral_angle" + 0.004*"detail_analysis" + 0.004*"rotate_phenylene_ring_deg" + 0.004*"function_positive_construction"'), (3, '0.044*"aring_resolution" + 0.009*"crystal_form" + 0.008*"aromatic_system" + 0.008*"crystal_symmetry" + 0.007*"molecular_replacement" + 0.006*"active_site" + 0.006*"unit_ce

### HDP Modeling

In [52]:


numtops = 100

hdp = HdpModel(corpus, dictionary, T=  numtops)

lda_model = hdp.suggested_lda_model()


C:\Users\pjz1\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\hdpmodel.py:222: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


NameError: name 'numTops' is not defined

In [57]:
i = 0

for k in range(numtops):
    print(lda_model.print_topic(k))
    if str(lda_model.print_topic(k)) == "":
        i += 1

print(i)


0.000*"mu_acid" + 0.000*"hps_hydrogen_bond" + 0.000*"molecular_crystal" + 0.000*"elastic_displacement_origin" + 0.000*"picolinate_ligand" + 0.000*"dimeric_ngal_crystal" + 0.000*"crystallochemical_role" + 0.000*"conformation_systematic_study" + 0.000*"volume_reduction" + 0.000*"hexaanion_minus_independent_centrosymmetric"
0.000*"protein_molecule" + 0.000*"symmetrical_tile" + 0.000*"herringbone_motif" + 0.000*"strong_c_h_pi" + 0.000*"magnetic_field_perpendicular_plane" + 0.000*"relatively_weak_magnetic_field" + 0.000*"structural_investigation" + 0.000*"n_butylammonium" + 0.000*"complete_chain" + 0.000*"delta_lambda"
0.000*"solid_helium_anneal_effect" + 0.000*"dispersion_surface" + 0.000*"arc_ni_presence_melt" + 0.000*"protein_limit_trypsin_digestion" + 0.000*"energy_dispersive" + 0.000*"angular_interaction" + 0.000*"convergent_beam" + 0.000*"archaeal_tim_surface" + 0.000*"substituent_low_effectivity" + 0.000*"one_cu_atom"
0.000*"paroxetine_hemihydrate_arrangement_stability" + 0.000*"prot

0.000*"sulfonyl_benzene_ring" + 0.000*"lie_spiro_c_atom" + 0.000*"different_crystallization_environment" + 0.000*"gene_expression" + 0.000*"h_datum_acquisition" + 0.000*"p_toluenesulfonate_anion" + 0.000*"paper_correspondence_address" + 0.000*"radiation_source" + 0.000*"correspond_size" + 0.000*"benzofuran_ring"
0.000*"overall_numb" + 0.000*"direct_comparison" + 0.000*"methoxy_o" + 0.000*"characteristic_torsion" + 0.000*"protein_domain_definition" + 0.000*"particular_emphasis" + 0.000*"glutamate_aspartate_residue" + 0.000*"intra_ligand_bridge_group" + 0.000*"previous_refinement" + 0.000*"dilute_silica"
0.000*"classical_protein_tyrosine" + 0.000*"mainly_ionic_interaction" + 0.000*"obtain_inclusion_crystal" + 0.000*"original_paper" + 0.000*"crystal_in_lattice" + 0.000*"residue_catalytic_acid" + 0.000*"direct_method" + 0.000*"ring_lang" + 0.000*"approximate_formula" + 0.000*"pattern_decomposition_method"
0.000*"phosphatidylinositide_pi_substrate" + 0.000*"n_c_bond_delocalization" + 0.000*

KeyboardInterrupt: 

In [59]:
print(hdp.hdp_to_lda()[0])

[1.92209116e-02 1.41664816e-02 1.24024268e-02 7.56196550e-03
 1.00789218e-02 9.86065819e-03 7.17395957e-03 5.30933128e-03
 8.37878798e-03 6.26207712e-03 6.16043181e-03 7.04477746e-03
 8.19584706e-03 7.79081995e-03 4.74463442e-03 8.11583898e-03
 4.99237941e-03 2.41108747e-03 3.57759029e-03 4.08208116e-03
 3.20183996e-03 3.37405786e-03 7.08039469e-03 3.89221532e-03
 3.33616307e-03 5.34472191e-03 3.96005922e-03 4.35245701e-03
 4.72550322e-03 4.64708717e-03 5.96457331e-03 7.33079313e-03
 5.44996664e-03 3.99178520e-03 6.99751993e-03 5.77130301e-03
 4.18191297e-03 3.31316151e-03 3.13282609e-03 4.45701412e-03
 4.71449303e-03 6.09788799e-03 4.62936577e-03 3.07548994e-03
 3.69855192e-03 3.20177384e-03 3.52135644e-03 4.26301955e-03
 2.92128538e-03 4.39217653e-03 4.42843145e-03 3.94544145e-03
 3.30937375e-03 4.19875357e-03 4.43512856e-03 2.12382384e-03
 3.58949092e-03 4.57893829e-03 4.26674435e-03 2.33869385e-03
 5.15515942e-03 2.81643390e-03 2.72286641e-03 2.81658657e-03
 4.53828696e-03 4.033840

In [58]:

top_words_per_topic = []
for t in range(lda_model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 10)])

terms = pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("topics.csv", mode='a', header=False)


KeyboardInterrupt: 

### Visualization

In [53]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.show(vis)

C:\Users\pjz1\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [08/Jul/2019 14:52:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2019 14:52:05] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2019 14:52:05] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2019 14:52:05] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2019 14:52:05] code 404, message Not Found
127.0.0.1 - - [08/Jul/2019 14:52:05] "GET /favicon.ico HTTP/1.1" 404 -



stopping Server...


### Document Tagging

In [8]:
i = 1

out =  csv.writer(open("output.csv", 'w'), lineterminator = '\n')
out.writerow(["prob", "topic", "docID"])

d2t = dict()
t2d = dict()


for ID in docIDs:
    
    print("Document " + str(i) + ": ")
    
    doc = docTokens[ID]
    
    store = list(lda_model[dictionary.doc2bow(doc)])
    
    print(store)
    
    for pair in store:
        
        
        out.writerow([pair[1], pair[0], ID])
        
        if ID in d2t:
            d2t[ID] += [pair]
        else:
            d2t[ID] = [pair]

        if pair[0] in t2d:
            t2d[pair[0]] += [(ID, pair[1])]
        else:
            t2d[pair[0]] = [(ID, pair[1])]

    
    i += 1

    

Document 1: 
[[(0, 0.5861223), (9, 0.08409163), (12, 0.08822708), (16, 0.18581915)], [(0, [0, 16, 12, 9]), (1, [0]), (2, [0, 16, 12, 9]), (3, [9, 0, 16]), (4, [0, 16, 12, 9]), (5, [12, 0]), (6, [16, 0]), (7, [0, 16, 12, 9]), (8, [16, 9, 0]), (9, [16]), (10, [0, 16, 12, 9]), (11, [0, 16, 12, 9]), (12, [0]), (13, [0, 16, 12, 9]), (14, [0, 16, 12, 9]), (15, [0]), (16, [0]), (17, [0, 16, 12, 9]), (18, [0])], [(0, [(0, 0.7118674), (9, 0.06155272), (12, 0.09556008), (16, 0.13097808)]), (1, [(0, 0.9999739)]), (2, [(0, 0.7118674), (9, 0.06155272), (12, 0.09556008), (16, 0.13097808)]), (3, [(0, 0.06687656), (9, 0.87177324), (16, 0.06120805)]), (4, [(0, 0.7118674), (9, 0.06155272), (12, 0.09556008), (16, 0.13097808)]), (5, [(0, 0.08904103), (12, 0.89683855)]), (6, [(0, 0.2721491), (16, 0.7278368)]), (7, [(0, 0.71187097), (9, 0.06155197), (12, 0.09555891), (16, 0.13097648)]), (8, [(0, 0.12663276), (9, 0.18712486), (16, 0.6862422)]), (9, [(16, 1.0000001)]), (10, [(0, 0.7118674), (9, 0.06155272), (

TypeError: unhashable type: 'list'

In [71]:
import operator


    
d2tWriter = csv.writer(open("d2t.csv", 'w'), lineterminator = '\n')
t2dWriter = csv.writer(open("t2d.csv", "w"), lineterminator = "\n")    



d2tWriter.writerow(["doc ID", "topic pairs"])
for ID in d2t:
    d2tWriter.writerow([ID, sorted(d2t[ID], key=operator.itemgetter(1), reverse=True)])
    
t2dWriter.writerow(["topic", "docID pairs"])
for topic in t2d:
    t2dWriter.writerow([topic]+ sorted(t2d[topic], key=operator.itemgetter(1), reverse=True))

### Searches

In [72]:
import sys
import urllib.request
from urllib.error import HTTPError


BASE_URL = 'http://dx.doi.org/'

def getTitle(doi):
    url = BASE_URL + doi
    req = urllib.request.Request(url)
    req.add_header('Accept', 'application/x-bibtex')
    try:
        with urllib.request.urlopen(req) as f:
            bibtex = f.read().decode()
        start = bibtex.find("title = {")
        end = bibtex.find("},", start)
        return bibtex[start + 9:end]
        
        
    except HTTPError as e:
        if e.code == 404:
            return('DOI not found.')
        else:
            return('Service unavailable.')

In [106]:



def search(docID, resistance):
    
    title =getTitle(docID)
    print("For the document: " + title)
    
    topics = d2t[docID]
    
    related = list()
    
    for topic in topics:
        
        if topic[1] > resistance:
            
            print("Topic " + str(topic[0]) + ": " + lda_model.print_topic(topic[0]))
            
            for newID in t2d[topic[0]]:
                if topic[1]*newID[1] > resistance:
                    related.append(getTitle(newID[0]))
    
    try:
        related.remove(title)
    except:
        pass
    print("We found: " + str(related))
    
search("10.1107/S0021889803000281", 0.93)

For the document: Small-angle neutron scattering by porous alumina membranes made of aligned cylindrical channels
Topic 15: 0.000*"chelate_molecule_n_atom" + 0.000*"bipyramidal_coordination_environment" + 0.000*"o_phenol_phenol" + 0.000*"crystalline_arrangement" + 0.000*"cylindrical_shape" + 0.000*"discrete_cation" + 0.000*"symmetry_relate_bmi_imidazole" + 0.000*"thiourea_unit" + 0.000*"m_acetate_ph_buff" + 0.000*"seed_achieve_method"
We found: ['Characterization of dislocations in protein crystals by means of synchrotron double-crystal topography', 'The local domain configuration in partially ordered {AuCu}3', 'X-ray near-field speckle: implementation and critical analysis', 'An Ultra-Low-Temperature Diffractometer Based on an 3He{\\textendash}4He Dilution Refrigerator used for Synchrotron-Radiation X-ray Diffractometry and Topography', '{CTD} Code: a Combinatorial Code for Eukaryotic Transcription']


Or, if we use cosine similarity.

In [23]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def get_cosine_sim(*strs): 
    vectors = [t for t in get_vectors(*strs)]
    return cosine_similarity(vectors)
    
def get_vectors(*strs):
    text = [t for t in strs]
    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()

def search(docID, resistance):
    
    title = getTitle(docID)
    print("For the document: " + title)
    
    str1 = " ".join(docTokens[docID])
    
    found = list()
    for ID in docIDs:
        tokens = docTokens[ID]
        str2 = " ".join(tokens)
        if get_cosine_sim(str1, str2)[0][1] > resistance:
            found.append(getTitle(ID))
    
    
    found.remove(title)
    print("We found: " + str(found))
    
search("10.1107/S0021889803000281", 0.07)

For the document: Small-angle neutron scattering by porous alumina membranes made of aligned cylindrical channels
We found: ['Structure of {PEP}{\\textendash}{PEO} block copolymer micelles: exploiting the complementarity of small-angle X-ray scattering and static light scattering', 'A new method of evaluating slit-smeared small-angle X-ray scattering data', 'First data acquired on the {extendedQ}-range small-angle neutron scattering ({EQ}-{SANS}) diffractometer at the Spallation Neutron Source', 'Comment on Misorientation-angle distribution of randomly oriented symmetric objects by Morawiec (1995)', 'The structure of pumice by neutron diffraction', 'Small-angle scattering curves of concentrated polymer solutions', 'Small-angle X-ray study of the three-dimensional collagen/mineral superstructure in intramuscular fish bone', 'Anomalous dispersion of small-angle scattering of horse-spleen ferritin at the {ironKabsorption} edge', 'Illustration of the anisotropic Porod law', 'A new method t